时间旅行

在典型的聊天机器人工作流程中，用户需要与机器人进行一次或多次交互才能完成一项任务。 记忆和人机交互机制会在图形状态中启用检查点，并控制未来的响应。

如果您希望用户能够从之前的回复开始，探索不同的结果，该怎么办？或者，如果您希望用户能够回放聊天机器人的工作以修复错误或尝试不同的策略（这在自主软件工程师等应用中很常见），该怎么办？

In [3]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from llm_layer import llm

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [4]:
'''向图表中添加步骤, 每个步骤都会在其状态历史记录中设置检查点'''
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================

[{'text': "I'll search for information about LangGraph using the Tavily search engine.", 'type': 'text'}, {'id': 'toolu_01RDJB4AxRATu9jdtTYhjeMS', 'input': {'query': 'What is LangGraph and how is it used in LLM applications'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01RDJB4AxRATu9jdtTYhjeMS)
 Call ID: toolu_01RDJB4AxRATu9jdtTYhjeMS
  Args:
    query: What is LangGraph and how is it used in LLM applications
================================= Tool Message =================================
Name: tavily_search

{"query": "What is LangGraph and how is it used in LLM applications", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "LangGraph Tutorial: What Is LangGraph and How to Use It?", "url": 

In [5]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

[{'text': 'Let me search for specific information about building autonomous agents with LangGraph!', 'type': 'text'}, {'id': 'toolu_01C2bnt6HQQ7Lg6kWirbEf9R', 'input': {'query': 'how to build autonomous agents with LangGraph examples tutorials'}, 'name': 'tavily_search', 'type': 'tool_use'}]
Tool Calls:
  tavily_search (toolu_01C2bnt6HQQ7Lg6kWirbEf9R)
 Call ID: toolu_01C2bnt6HQQ7Lg6kWirbEf9R
  Args:
    query: how to build autonomous agents with LangGraph examples tutorials
================================= Tool Message =================================
Name: tavily_search

{"query": "how to build autonomous agents with LangGraph examples tutorials", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "How to Build AI Agents with La

In [6]:
'''重放完整的历史'''
# 检查点会保存图表中每一步的执行情况。这涵盖了所有调用 ，因此您可以回溯整个线程的历史记录。
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  12 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  11 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  10 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  9 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  8 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  7 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------

In [7]:
'''从检查点恢复'''
print(to_replay.next)
print(to_replay.config)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f05328a-63c2-6878-8005-2e0766216e7c'}}


In [8]:
'''加载某个时刻的状态'''
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

Based on the search results, I can provide you with a comprehensive overview of LangGraph:

LangGraph is an open-source AI agent framework that's part of the LangChain ecosystem. Here are the key points about LangGraph:

1. Purpose and Core Functionality:
- It's designed to build, deploy, and manage complex generative AI agent workflows
- Provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner
- Uses graph-based architectures to model and manage relationships between various components of an AI workflow

2. Key Features:
- State Management: Offers built-in support for persistent checkpoints and state tracking
- Transparency: Illuminates processes within an AI workflow, allowing full visibility of agent states
- Multi-agent Coordination: Enables management of multiple AI agents working together
- Cycle Support: Handles recursive or cyclical proc